In [62]:
# export
import Financial_Indicators as fi
import numpy as np
import pandas as pd
import pickle

In [2]:
# export
def load_data():
    file = open("df_all.pkl","rb")
    df_all = pickle.load(file)
    return df_all

In [21]:
# export
def f_temp(x, thr):
    if x >= thr:
        return 1
    elif x <= 1- thr:
        return -1
    else:
        return 0


def f_temp3(x, thr):
    idx = np.argmax(x)
    if (idx == 0) & (x[idx] >= thr):
        return -1
    elif (idx == 1) & (x[idx] >= thr):
        return 0
    elif (idx == 2) & (x[idx] >= thr):
        return 1
    else:
        return -99    
    
    
def f_classif_report(clf, X, y, thr = 0.5):
    orig_len = len(y)
    
    if len(clf.predict_proba(X)[0]) == 3:
        y_pred = [f_temp3(x, thr) for x in clf.predict_proba(X)]
        np_array = np.array(y_pred)
        item_index = np.where((np_array==-1) | (np_array==1) | (np_array==0) )
        item_index = list(item_index[0])
        y_pred = [y_pred[x] for x  in item_index ]
        y_list = [x[0] for x in  y.values]
        y = [y_list[x] for x  in item_index ]
    elif len(clf.predict_proba(X)[0]) == 2:
        y_pred = [f_temp(x[1], thr) for x in clf.predict_proba(X)]
        np_array = np.array(y_pred)
        item_index = np.where((np_array==-1) | (np_array==1) )
        item_index = list(item_index[0])
        y_pred = [y_pred[x] for x  in item_index ]
        y_list = [x[0] for x in  y.values]
        y = [y_list[x] for x  in item_index ]
    
    last_len = len(y)
    
    loss = 1 - (last_len/orig_len)
    print("Lost on dataset: {x}".format(x = loss ) )  
    print(classification_report(y, y_pred))
    print(confusion_matrix(y, y_pred))
    return y, y_pred, loss


def f_prep_df_to_ML(df, is_train = True):
    df_tmp = df.copy()
    if is_train:
        df_tmp.dropna(inplace = True)
    y = df_tmp.loc[:, ["Target"]]
    X = df_tmp.loc[:, [ x for x in df_tmp.columns if x not in (["Target" 
                                                                ,"coin" 
                                                                ,"next1price" 
                                                                ,"next2price" 
                                                                ,"next3price" 
                                                                ,"price"
                                                               # ,"is_day_end"
                                                               # ,"last_time"
                                                               ])  ]]

    return X, y 


In [68]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import SVC

df_all_coins = load_data()

model_dict = {}
list_all = []
for coin in df_all_coins.coin.unique():
#for coin in ['DOGEUSDT']: #DOGEUSDT, LUNAUSDT, XRPUSDT,UNIUSDT, FILUSDT
    df_tmp = df_all_coins.loc[df_all_coins.coin==coin, : ]
    
    #df_tmp = df_all_coins.loc[(df_all_coins.coin==coin) & (df_all_coins.Target != 0), : ]
    X, y = f_prep_df_to_ML(df_tmp)
    
    clf = RandomForestClassifier(n_estimators=10 ,random_state=0, max_depth = 5, class_weight = "balanced")
    #clf.fit(X , y)
    clf = OneVsRestClassifier(clf).fit(X, y)

    del df_tmp
    
    thr = 0.4
    is_model_valid = False
    while thr < 0.91:
        try:
            y_true, y_pr, loss = f_classif_report(clf, X, y, thr)
            f1 = f1_score(y_true, y_pr, average='macro')
            print("{coin} model trial, f1_Score: {f1_Score} and loss is: {loss}".format(coin =coin ,f1_Score = str(f1), loss = str(loss) ))
            if (f1 > 0.8) & (loss <= 0.7):
                is_model_valid = True
                print("model is OK")
                break;
            #thr = thr + 0.02       
        except Exception as e:
            print(str(e))
            pass
        thr = thr + 0.02
    
    if is_model_valid:
        model_dict.update ({coin: [clf, thr]})
    #graph for feature importance

    #from sklearn.metrics import f1_score
    #f1_score(y, y_pred, average='macro')    
    print('**********************')

pickle_out = open("model_dict.pkl","wb")
pickle.dump(model_dict, pickle_out)
xpickle_out.close()    

Lost on dataset: 0.077141504969238
              precision    recall  f1-score   support

          -1       0.68      0.72      0.70       382
           0       0.85      0.87      0.86      1119
           1       0.77      0.69      0.73       449

    accuracy                           0.80      1950
   macro avg       0.77      0.76      0.76      1950
weighted avg       0.80      0.80      0.80      1950

[[274  77  31]
 [ 80 977  62]
 [ 49  91 309]]
DOGEUSDT model trial, f1_Score: 0.7624559474188062 and loss is: 0.077141504969238
Lost on dataset: 0.11784193090392803
              precision    recall  f1-score   support

          -1       0.72      0.73      0.72       361
           0       0.86      0.89      0.88      1075
           1       0.79      0.71      0.75       428

    accuracy                           0.82      1864
   macro avg       0.79      0.78      0.78      1864
weighted avg       0.82      0.82      0.82      1864

[[263  69  29]
 [ 64 961  50]
 [ 40  8

NameError: name 'xpickle_out' is not defined

In [69]:
X_c = X.copy()

X.loc[:,"proba_1"] = [x[0] for x in clf.predict_proba(X_c)] 
X.loc[:,"proba_0"] = [x[1] for x in clf.predict_proba(X_c)] 
X.loc[:,"proba1"] = [x[2] for x in clf.predict_proba(X_c)] 
X


,vol,tradefreq,last_time,is_day_end,rsi14,rsi28,rsi42,rsi48,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,macd12_ratio,signal12_ratio,proba_1,proba_0,proba1
dt,,,,,,,,,,,,,,,,,,,,
2020-05-20 16:00:00,3.213652e+07,396.0,0,0,43.887082,45.630034,46.065387,46.071442,46.417016,-0.000004,4.175957e-07,-0.000004,-0.000008,-0.000005,-0.000004,-0.001595,0.000167,0.050994,0.860167,0.088839
2020-05-20 20:00:00,9.854109e+06,198.0,0,0,44.988429,46.097334,46.347643,46.313010,46.572835,-0.000005,-6.875345e-07,-0.000004,-0.000009,-0.000006,-0.000002,-0.002043,-0.000275,0.054025,0.857420,0.088555
2020-05-21 00:00:00,2.691318e+07,304.0,0,1,43.420544,45.405954,45.927031,45.953874,46.339387,-0.000007,-1.855610e-06,-0.000005,-0.000009,-0.000007,-0.000002,-0.002620,-0.000745,0.081205,0.832784,0.086011
2020-05-21 04:00:00,1.397286e+07,205.0,0,0,42.716688,45.098003,45.740347,45.794652,46.236205,-0.000008,-3.048232e-06,-0.000005,-0.000010,-0.000008,-0.000001,-0.003142,-0.001225,0.081205,0.832784,0.086011
2020-05-21 08:00:00,5.225856e+07,799.0,0,0,49.413504,47.877998,47.401235,47.211184,47.141320,-0.000007,-3.780853e-06,-0.000003,-0.000006,-0.000008,0.000001,-0.002668,-0.001503,0.185287,0.716260,0.098452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-07 00:00:00,2.395894e+09,1205898.0,0,1,56.768092,61.378811,61.868631,61.834136,61.508787,0.040594,5.135131e-02,-0.010758,0.003781,0.011519,-0.007739,0.070816,0.089582,0.390681,0.023956,0.585364
2021-05-07 04:00:00,2.090216e+09,1161210.0,0,0,64.451150,65.495248,64.912136,64.604822,63.632447,0.040678,4.921668e-02,-0.008539,0.008650,0.010372,-0.001721,0.064321,0.077823,0.316186,0.024977,0.658837
2021-05-07 08:00:00,1.047569e+09,642152.0,0,0,62.925160,64.590615,64.246526,64.003548,63.184506,0.039689,4.731122e-02,-0.007622,0.010224,0.010313,-0.000088,0.063511,0.075708,0.287030,0.022674,0.690296


In [48]:
  #2021-05-08 16:00:00

from datetime import datetime

datetime_str = '2021-05-07 08:00:00'
dt = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')

X.loc[dt, :]

vol               1.047569e+09
tradefreq         6.421520e+05
last_time         0.000000e+00
is_day_end        0.000000e+00
rsi14             6.292516e+01
rsi28             6.459062e+01
rsi42             6.424653e+01
rsi48             6.400355e+01
rsi72             6.318451e+01
macd12            3.968941e-02
signal12          4.731122e-02
hist12           -7.621818e-03
macd6             1.022438e-02
signal6           1.031277e-02
hist6            -8.838446e-05
macd12_ratio      6.351118e-02
signal12_ratio    7.570765e-02
Name: 2021-05-07 08:00:00, dtype: float64

In [25]:
model_dict

{'DOGEUSDT': [OneVsRestClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                       max_depth=5,
                                                       n_estimators=10,
                                                       random_state=0)),
  0.4600000000000001]}

In [22]:
from exporter import export
export("MLTrain.ipynb","MLTrain.py")